# Behind the Scenes of ChatGPT
---
[Pico GPT](https://github.com/jaymody/picoGPT/blob/29e78cc52b58ed2c1c483ffea2eb46ff6bdec785/gpt2_pico.py#L43-L58). 코드를 단계적으로 이해해 보겠습니다. [Original Colab Code](https://colab.research.google.com/drive/1yEA-lkxJm5eSO6LkcIvkdWz_bE_q2LQj?usp=sharing)
- Author: [Jay Mody](https://jaykmody.com)
- Code: [picoGPT](https://github.com/jaymody/picoGPT)
- Blog Post: [GPT in 60 Lines of NumPy](https://jaykmody.com/blog/gpt-from-scratch)

# Input / Output
---
가장 간단한 자기회귀 (autoregressive) 모델은 다음과 같습니다.

```python
for _ in range(n_tokens_to_generate):
    next_word = gpt(prompt)
    prompt = prompt + next_word
```

자기회귀 모델에서 사용된 `gpt` 함수는 다음과 같은 형태를 갖고 있습니다.

```python
def gpt(prompt: str) -> str:
    next_word = # beep boop neural network magic
    return next_word
```

하지만 신경망 내부 에서는 숫자로 임베딩한 값을 사용하기 때문에, 다음과 같이 변경 했습니다.

```python
def gpt(inputs: list[int]) -> list[list[float]]:
    # inputs has shape [n_seq]
    # output has shape [n_seq, n_vocab]
    output = # beep boop neural network magic
    return output
```

# 1 Input
## 1-1 Encoding
- One Hot Encoding : Encoding & Decoding

In [1]:
# 정수 값으로 Token 을 임베딩 하는 예제 입니다.
inputs = [1,     0,    2,      4,     6]
vocab  = ["all", "not", "heroes", "the", "wear", ".", "capes"]
for id in inputs:
    print(vocab[id])

not
all
heroes
wear
capes


In [2]:
# 인코더와 디코더를 클래스로 작성하였습니다
class WhitespaceTokenizer:
    def __init__(self, vocab):
        self.token_to_id = {token: i for i, token in enumerate(vocab)}
        self.id_to_token = {i: token for i, token in enumerate(vocab)}

    def encode(self, text):
        tokens = text.split(" ")
        ids = [self.token_to_id[token] for token in tokens]
        return ids

    def decode(self, ids):
        tokens = [self.id_to_token[id] for id in ids]
        text = " ".join(tokens)
        return text

# 문장을 Token 으로 분리 후, 숫자로 Embadding 하는 클래스
# .encode() : converts a str -> list[int]
# .decode() : back a list[int] -> str
tokenizer = WhitespaceTokenizer(vocab)
ids = tokenizer.encode("not all heroes wear")
text = tokenizer.decode(ids)
ids, text

([1, 0, 2, 4], 'not all heroes wear')

In [3]:
# 맨 앞에서 장의한 `vocab` 을 활용하여, 문장의 Token 을 Mapping 합니다.
tokens = [vocab[i] for i in ids]
tokens

['not', 'all', 'heroes', 'wear']

## 1-2 Input Errors
오류가 발생하는 상황들을 살펴보겠습니다.

In [4]:
def check_input_error(texts:str):
    try:
        print(tokenizer.encode(texts))
    except KeyError as E:
        print(f"KeyError : {E}")

# Q1 `vocab` 에 없는 단어를 입력하면 어떻게 될까?
check_input_error("not all heroes wear shoes")
# Q2 `vocab` 에 없는 multiple spaces 가 있으면 어떻게 될까?
check_input_error("not all heroes    wear")
# Q3 '' 을 입력하면 어떻게 될 까?
check_input_error("")
# Q4 `punctuation` 기호가 포함되어 있으면 어떻게 될 까?
check_input_error("not all heroes wear capes!")

KeyError : 'shoes'
KeyError : ''
KeyError : ''
KeyError : 'capes!'


## 1-3 Input 내용을 마치며
이처럼 공백을 기준으로 Token 을 나누는 방법 보다는 [Byte-Pair Encoding](https://huggingface.co/course/chapter6/5?fw=pt) 또는 [WordPiece](https://huggingface.co/course/chapter6/6?fw=pt) 와 같은 고급화 기법을 주로 활용 합니다. 고급화 기법 이라도 동작 원리는 앞에서 살펴본 내용과 동일 합니다.
1. 문자열을 입력 받습니다.
2. **<span style="color:orange">토큰화 도구</span>** 를 사용하여 문자열을 토큰이라는 작은 조각으로 분해합니다.
3. 토큰을 정수로 매핑하기 위해, **<span style="color:orange">어휘 사전</span>** 을 사용합니다.

그렇다면 **<span style="color:orange">어휘 사전</span>** 에 무엇이 들어갈지 어떻게 결정할까요? **Tokenizer** 에서는 여러 **<span style="color:orange">최적의 어휘가 무엇인지 알아내는 '훈련' 프로세스</span>** 도 포함되어 있습니다.

<br/>

# 2 OutPut
## 2-1 Output
출력 형태는 2차원의 배열 (`output[i][j]`) 형태를 갖습니다. `vocab[j]` 값의 의미는 바로 다음에 연결되는 `inputs[i+1]` Token 을 예측한 확률 입니다.
```python
output = gpt(inputs)
#              ["all", "not", "heroes", "the", "wear", ".", "capes"]
# output[0] =  [0.75    0.1     0.0       0.15    0.0   0.0    0.0  ]
# "not" 보단 "all" 의 가능성을 더 높게 예측

#              ["all", "not", "heroes", "the", "wear", ".", "capes"]
# output[1] =  [0.0     0.0      0.8     0.1    0.0    0.0   0.1  ]
# ["not", "all"] 시퀀스 부분을 입력할 때, 모델은 "heroes" 를 가장 높게 예측

#              ["all", "not", "heroes", "the", "wear", ".", "capes"]
# output[-1] = [0.0     0.0     0.0     0.1     0.0    0.05  0.85  ]
# 전체 시퀀스인 ["not", "all", "heroes", "wear"] 을 입력하면, "capes" 확률이 가장 높음
```

In [5]:
vocab  = ["all", "not", "heroes", "the", "wear", ".", "capes"]
inputs = [1, 0, 2, 4] # "not" "all" "heroes" "wear"

import numpy as np
output = np.array([
    [0.75, 0.1, 0.0, 0.15, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.8, 0.1, 0.0, 0.0, 0.1],
    # ...
    [0.0, 0.0, 0.0, 0.1, 0.0, 0.05, 0.85],
])
next_id = np.argmax(output[-1]) # 마지막 배열에서 가장 높은 값의 인덱스
next_id, vocab[next_id]         # 해당 인덱스에서 임베딩하는 Token 값을 출력

(6, 'capes')

## 2-2 Output 내용을 마치며
이처럼 **<span style="color:orange">해당 시점에서 가장 높은 확률을 선택</span>** 하는 방법을 **greedy decoding** 또는 **greedy sampling** 이라고 합니다.

이 방법은 1등과 2등이 미묘한 차이를 갖는 경우라면, 2등이 정답일 가능성도 고려해야 합니다. 이와같은 상황에 적용할 수 있도록 **분포** 값을 적용하는 [Beam Search](https://blog.naver.com/PostView.naver?blogId=sooftware&logNo=221809101199) 또는 [top-p and top-k](https://docs.cohere.ai/docs/controlling-generation-with-top-k-top-p) 등의 방법이 있습니다.

이번 과정에서는 빠르게 진행 가능한 **greedy sampling** 을 활용해 보겠습니다.

앞에서 살펴본 자기회귀 (autoregressive) 모델에서 `np.argmax()` 를 추가 합니다.

```python
for _ in range(n_tokens_to_generate):
    output = gpt(ids)
    next_id = np.argmax(output[-1])
    ids.append(next_id)
```

이처럼 GPT 에서는 바로 이어지는 다음 단어만 예측하면 되는데, 여러개의 확률 벡터를 입력받는 이유는 왜일까요? 마지막 추론 과정에서는 `Output` 챕터에서 살펴본 것처럼 마지막 확률값(`output[-1]`) 을 활용하지만 전체적인 훈련 과정은 모든 출력값을(`outputs[i]` 값은 `ids[i+1]` 인덱스 값) 활용 합니다. 

이렇게 하면 각각의 Token 마다 **Forward Passes** 를 수행하지 않고, 모든 Token 에 대해서 **병렬로 계산** 을 할 수 있게되어 GPT 훈련을 매우 효율적으로 실행할 수 있습니다.

<br/>

# 3 GPT 구현하기
- Load Tokenizer, Hyperparamters, and Parameters
## 3-1 Tokenizer
[BPE tokenizer](https://huggingface.co/course/chapter6/5?fw=pt) 은 GPT2 에서 사용되고 있습니다.
```python
! git clone https://github.com/jaymody/picoGPT
import sys
sys.path.append("picoGPT")
from gpt2 import main
output = main(
    prompt="Alan Turing theorized that computers would one day become",
    model_size="124M",
    n_tokens_to_generate=8
)
output # ' the most powerful machines on the planet.'
```

In [6]:
%reset -f

In [8]:
import os
import sys
import json
import tensorflow as tf
sys.path.append("./gits/picoGPT")

from encoder import Encoder
from utils import download_gpt2_files, load_gpt2_params_from_tf_ckpt

def get_encoder(model_name, models_dir):
    with open(os.path.join(models_dir, model_name, "encoder.json"), "r") as f:
        encoder = json.load(f)
    with open(os.path.join(models_dir, model_name, "vocab.bpe"), "r", encoding="utf-8") as f:
        bpe_data = f.read()
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split("\n")[1:-1]]
    return Encoder(encoder=encoder, bpe_merges=bpe_merges)

def load_encoder_hparams_and_params(model_size, models_dir):
    assert model_size in ["124M", "355M", "774M", "1558M"]
    model_dir = os.path.join(models_dir, model_size)
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)

    # download files if necessary
    if not tf_ckpt_path:
        os.makedirs(model_dir, exist_ok=True)
        download_gpt2_files(model_size, model_dir)
        tf_ckpt_path = tf.train.latest_checkpoint(model_dir)

    encoder = get_encoder(model_size, models_dir)
    hparams = json.load(open(os.path.join(model_dir, "hparams.json")))
    params  = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, hparams)
    return encoder, hparams, params

# from utils import load_encoder_hparams_and_params
tokenizer, hparams, params = load_encoder_hparams_and_params(
    model_size="124M",
    models_dir="data"
)
ids = tokenizer.encode("Not all heroes wear capes.")
ids, tokenizer.decode(ids)

([3673, 477, 10281, 5806, 1451, 274, 13], 'Not all heroes wear capes.')

Using the vocabulary of the tokenizer (stored in `encoder.decoder`), we can take a peek at what the actual tokens look like:

In [ ]:
[tokenizer.decoder[i] for i in ids]

['Not', 'Ġall', 'Ġheroes', 'Ġwear', 'Ġcap', 'es', '.']

Notice, sometimes our tokens are words (e.g. `Not`), sometimes they are words but with a space in front of them (e.g. `Ġall`, the [`Ġ` represents a space](https://github.com/karpathy/minGPT/blob/37baab71b9abea1b76ab957409a1cc2fbfba8a26/mingpt/bpe.py#L22-L33)), sometimes there are part of a word (e.g. capes is split into `Ġcap` and `es`), and sometimes they are punctuation (e.g. `.`).

One nice thing about BPE is that it can encode any arbitrary string. If it encounters something that is not present in the vocabulary, it just breaks it down into substrings it does understand:

In [ ]:
[tokenizer.decoder[i] for i in tokenizer.encode("zjqfl")]

['z', 'j', 'q', 'fl']

In [ ]:
len(tokenizer.decoder)

50257

## Hyperparameters

`n_vocab`: Number of tokens in our vocabulary

`n_layer`: Number of layers (determines the "depth" of the network)

`n_embd`: Embedding dimension (determines the "width" of the network)

`n_ctx`: Maximum possible sequence length of the input

`n_head`: Number of attention heads (`n_embd` must be divisible by `n_head`)


In [ ]:
hparams

{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}

## Parameters

In [ ]:
params.keys()

dict_keys(['blocks', 'ln_f', 'wpe', 'wte'])

In [ ]:
params["wte"]

array([[-0.11010301, -0.03926672,  0.03310751, ..., -0.1363697 ,
         0.01506208,  0.04531523],
       [ 0.04034033, -0.04861503,  0.04624869, ...,  0.08605453,
         0.00253983,  0.04318958],
       [-0.12746179,  0.04793796,  0.18410145, ...,  0.08991534,
        -0.12972379, -0.08785918],
       ...,
       [-0.04453601, -0.05483596,  0.01225674, ...,  0.10435229,
         0.09783269, -0.06952604],
       [ 0.1860082 ,  0.01665728,  0.04611587, ..., -0.09625227,
         0.07847701, -0.02245961],
       [ 0.05135201, -0.02768905,  0.0499369 , ...,  0.00704835,
         0.15519823,  0.12067825]], dtype=float32)

In [ ]:
import numpy as np

def param_count(d):
    if isinstance(d, np.ndarray):
        return np.size(d)
    elif isinstance(d, list):
        return sum(param_count(v) for v in d)
    elif isinstance(d, dict):
        return sum(param_count(v) for v in d.values())
    else:
        ValueError("uh oh")

param_count(params)

124439808

## GPT Function

Import the `gpt2` function from picoGPT.

In [ ]:
from gpt2 import gpt2

Let's inspect the function signature.

In [ ]:
help(gpt2)

Help on function gpt2 in module gpt2:

gpt2(inputs, wte, wpe, blocks, ln_f, n_head)



Notice that the function signature input includes some extra stuff in addition to `inputs`:

* `wte`, `wpe`, `blocks`, and `ln_f` the parameters of our model.
* `n_head` is a hyperparameter that is needed during the forward pass.

So, we can call our gpt2 function as such:

In [ ]:
input_ids = tokenizer.encode("The apple doesn't fall far from the")
input_ids

[464, 17180, 1595, 470, 2121, 1290, 422, 262]

In [ ]:
output = gpt2(input_ids, **params, n_head=hparams["n_head"])
output.shape

(8, 50257)

In [ ]:
next_id = np.argmax(output[-1])
next_id

5509

In [ ]:
tokenizer.decoder[next_id]

'Ġtree'

## Final Implementation

In [ ]:
%reset -f

In [ ]:
from tqdm import tqdm
import numpy as np

from gpt2 import gpt2
from utils import load_encoder_hparams_and_params

def generate(inputs, params, n_head, n_tokens_to_generate):
    # auto-regressive decode loop
    for _ in tqdm(range(n_tokens_to_generate), "generating"):
        # model forward pass
        logits = gpt2(inputs, **params, n_head=n_head)

        # greedy sampling
        next_id = np.argmax(logits[-1])

        # append prediction to input
        inputs.append(int(next_id))

    # only return generated ids
    output_ids = inputs[len(inputs) - n_tokens_to_generate :]
    return output_ids


def main(prompt: str, n_tokens_to_generate: int = 40, model_size: str = "124M", models_dir: str = "models"):
    # load encoder, hparams, and params from the released open-ai gpt-2 files
    tokenizer, hparams, params = load_encoder_hparams_and_params(model_size, models_dir)

    # encode the input string using the BPE tokenizer
    input_ids = tokenizer.encode(prompt)

    # make sure we are not surpassing the max sequence length of our model
    assert len(input_ids) + n_tokens_to_generate < hparams["n_ctx"]

    # generate output ids
    output_ids = generate(input_ids, params, hparams["n_head"], n_tokens_to_generate)

    # decode the ids back into a string
    output_text = tokenizer.decode(output_ids)

    return output_text

In [ ]:
main(
    "Alan Turing theorized that computers would one day become",
    n_tokens_to_generate=8,
    model_size="124M",
)

generating: 100%|██████████| 8/8 [00:05<00:00,  1.60it/s]


' the most powerful machines on the planet.'

# Next Steps?
---
See the full [blog post](https://jaykmody.com/blog/gpt-from-scratch/).